In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

from functions import import_imagedata, get_data, label_oh_tf,  Unwrap, ImageProcessor, IDSWDataSetLoader
from fns4wandb import build_optimizer, set_optimizer, hp_sweep, train_model, train_log, log_test_score, set_lossfn, pipeline, test_loop_batch, batch_loop
from loop_fns import res_loop, test_loop,test_loop_res

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from torchvision.models import vgg16
from torchvision.models import resnet101

import cv2

from torch.utils.data import DataLoader
import wandb
from fns4wandb import train_log, build_optimizer
from copy import deepcopy
from tqdm import tqdm

import random
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
import wandb
from fns4wandb import train_log, build_optimizer, hp_sweep
from copy import deepcopy
from tqdm import tqdm

import pickle
from fns4wandb import set_lossfn

In [2]:
#!pip install tensorflow
#!pip install keras
#import keras
from keras.datasets import mnist

2024-02-14 14:50:06.049436: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-14 14:50:06.077167: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-14 14:50:06.077186: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-14 14:50:06.077923: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-14 14:50:06.083307: I tensorflow/core/platform/cpu_feature_guar

In [3]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"

In [4]:
model_vgg16 = vgg16(weights="IMAGENET1K_V1")
#model_vgg16.features.pop(0)
model_vgg16.classifier.pop(6)
model_vgg16.to(device)
#print(model_vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [5]:
def metrics(label, prediction): #TypeError: Singleton array tensor(3) cannot be considered a valid collection.
    
    label= np.array(label.cpu())

    predictions_np = prediction.cpu().detach().numpy()
    #y_pred' parameter of f1_score must be an array-like or a sparse matrix. Got 7 instead.
    predicted_classes = np.argmax(predictions_np, axis=0)
    #print('metrics Label:   ', label)
    #print('metrics prediction   ', predicted_classes)
    #avg_f1_score = f1_score(label, predictions_np, average='macro')
    acc = accuracy_score(label, predicted_classes)
    
    return acc

In [6]:
def preprocess_im(img):
    IP = ImageProcessor(device='cpu')
    if isinstance(img, str):
        img = cv2.imread(img_path) #
    #imgx = img.shape[0]
    #imgy = img.shape[1]
    #img = img.reshape(imgx, imgy,1)
    #img = IP.blank_padding(img, (224,224))
    img = IP.to_tensor(img).to(device)
    return img

In [7]:
# get len of 

"""file_path = r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/'
random_seed =1
img_len = len(os.listdir(file_path))


#print(ids[4])
x, y = import_imagedata(file_path)

x_train, x_test, y_train, y_tests = train_test_split(x,y, test_size=0.2, train_size=0.8,
                                 random_state=random_seed, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.1, train_size=0.8,
                                 random_state=random_seed, shuffle=True)"""


"file_path = r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/'\nrandom_seed =1\nimg_len = len(os.listdir(file_path))\n\n\n#print(ids[4])\nx, y = import_imagedata(file_path)\n\nx_train, x_test, y_train, y_tests = train_test_split(x,y, test_size=0.2, train_size=0.8,\n                                 random_state=random_seed, shuffle=True)\nx_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.1, train_size=0.8,\n                                 random_state=random_seed, shuffle=True)"

In [8]:
wandb.login()

wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [9]:
title = f'IDSW_VGG16_fine_MNIST_testteest_290124'
save_dict = {'Run' : title,
            'Current_Epoch': 0}
                #r'/its/home/nn268/antvis/optics/
save_location = r'pickles/'#pickles

In [10]:
save_dict['Run']

'IDSW_VGG16_fine_MNIST_testteest_290124'

In [11]:
class Squeeze(nn.Module):
    def __init__(self):
        super(Squeeze, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        x = x.squeeze(0)
        return x

In [12]:

"""run_title = title#"IDSW_vgg_fine_MNISTtest_122023"

config = {
    'method': 'random',
    'metric':{
        'goal': 'minimize',
        'name': 'val_loss'},
    'parameters': {
        #'dropout':{
        #    'values': [0.5, 0.4, 0.3]
        #},
        'epochs':{
            'value': 20
        },

        'first_lin_lay':{
            'values':[248832]
        },
        'optimizer': {
            'values': ['adam']
        },
        'learning_rate': {
                # a flat distribution between 0 and 0.1
                'distribution': 'log_uniform_values',
                'min': 1e-6,
                'max': 1e-3
            },
        'loss_fn': {
            'values': ['CrossEntropy', 'MSE'] #'MSE', 
        },
        'data_set':{
            'values':['Augmented']
        },
            'scheduler': {
            'values': [0.2] #0.1,0.2, 0.01, 0.001
        },
        'ks': {
            'values': [(3,5)]
        },
        'channels':{
            'values': [3]
        },
        'weight_decay': {
            'values' :[0.5]},
        'dropout' :{
            'values': [0.2]
        },
        'num_classes': {
            'values': [10]
        },
        'model_name' : {'values': ['vgg16net_mlp']},
        'channels' : {'values': [3]},
        'image_path': {
            'values': [r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/']
        }
        }
    }
sweep_id = wandb.sweep(config, project=f"{run_title}")
"""
col_dict = {
    'colour': 'colour',
    'size': [57,15],
    'padding': 5,
    'model_size': 'vgg16'
}

config = dict(
    epochs= 120, #30, 
    learning_rate =1e-5,
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    ks = (3,5),
    scheduler=0.2,
    f_lin_lay = 2048,#8192,#5958,#65536,#8192,#7168, #1024*7 = 7168
    dataset = 'IDSW_Aug',
    loss_fn = 'CrossEntropy',
    model = "vgg16"
    
)


title = f"IDSWAug_vgg16_e120_{col_dict['size']}_140224"


In [13]:
class PrintLayer(nn.Module):
    def __init__(self):
        super(PrintLayer, self).__init__()
    
    def forward(self, x):
        # Do your print / debug stuff here
        print(x.shape)
        return x

In [14]:
"""from keras.preprocessing.image import img_to_array, array_to_img
from functions import ImageProcessor

IP= ImageProcessor(device)

def preprocessMNIST():
    #load data
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_val,y_train,  y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True)
    #print(x_train.shape,x_test.shape, x_val.shape)
    #print(len(x_train))
    #print('1')
    #IP.view(x_train[0],5)


    def convert(data): # convert greyscale image to a 3d gray image
        data_list = []
        for i in data:
            i = cv2.resize(i, (48,48))
            i = cv2.cvtColor(i, cv2.COLOR_GRAY2BGR)
            #print(type(i))
            data_list.append(i)
        data = np.array(data_list)
        return data


    x_train = convert(x_train)
    x_val = convert(x_val)
    x_test = convert(x_test)
    #print('post convert',x_train[0].shape)
    
    # normalise pixel vals
    x_train = [i/ 255 for i in x_train]
    x_test = [i/ 255 for i in x_test]
    x_val = [i/ 255 for i in x_val]
    
    # ensure array
    x_train= np.array(x_train)
    x_test= np.array(x_test)
    x_val= np.array(x_val)
    #print(type(x_train))
    #print('norm \n', x_train[0].shape, type(x_train),'\n', len(x_train))
    #IP.view(x_train[0],5)

    # convert to float 32
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_val = x_val.astype('float32')
    #print('flaot', x_train.shape, type(x_train), '\n')
    #IP.view(x_train[0],5)
    return x_train,y_train, x_val, y_val,  x_test, y_test"""

"from keras.preprocessing.image import img_to_array, array_to_img\nfrom functions import ImageProcessor\n\nIP= ImageProcessor(device)\n\ndef preprocessMNIST():\n    #load data\n    (x_train, y_train), (x_test, y_test) = mnist.load_data()\n    x_train, x_val,y_train,  y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True)\n    #print(x_train.shape,x_test.shape, x_val.shape)\n    #print(len(x_train))\n    #print('1')\n    #IP.view(x_train[0],5)\n\n\n    def convert(data): # convert greyscale image to a 3d gray image\n        data_list = []\n        for i in data:\n            i = cv2.resize(i, (48,48))\n            i = cv2.cvtColor(i, cv2.COLOR_GRAY2BGR)\n            #print(type(i))\n            data_list.append(i)\n        data = np.array(data_list)\n        return data\n\n\n    x_train = convert(x_train)\n    x_val = convert(x_val)\n    x_test = convert(x_test)\n    #print('post convert',x_train[0].shape)\n    \n    # normalise pixel vals\n    x_train = [i/ 255 for i i

In [15]:
def plot_grad_flow(model):

    #### This function, used specifically for the visualisation of gradient flow, was found in the following stackoverflow thread:
    # https://stackoverflow.com/questions/70394788/pytorch-adam-optimizer-dramatically-cuts-gradient-flow-compared-to-sgd
    #model = deepcopy(model).to('cpu')
    #print(model)
    model = model.to('cpu')
    named_parameters = model.named_parameters()
    #print(named_parameters)
    
    ave_grads = []
    max_grads= []
    layers = []
    for n, p in named_parameters:
        #print(n,p)
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            #print('gradddssss', p.grad)
            #print('p: ', p)
            ave_grads.append(p.grad.abs().mean())
            max_grads.append(p.grad.abs().max())
    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom = -0.001, top=1) #  top=0.02) # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    return model.to(device)

In [16]:
from architectures import PrintLayer, Flattern
def train(device,col_dict, save_dict, config=None):
    # lists for save dict
    t_loss_list = []
    v_loss_list =[]
    t_predict_list = []
    t_label_list = []
    v_predict_list = []
    v_label_list = []
    t_accuracy_list= []
    v_accuracy_list= []
    
    with wandb.init(config=config):
        config = wandb.config

        x_train, y_train, x_val, y_val, x_test, y_test = get_data(file_path= r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/', seed= random.randint(0, 50))
        
        
        #x_train_loader = IDSWDataSetLoader(x_train, y_train, col_dict=col_dict, device=device) #x_train, y_train, col_dict=col_dict, device=device
        #train_loader = DataLoader(x_train_loader, shuffle=True, batch_size=4)
        #x_test_loader = IDSWDataSetLoader(x_test, y_test, col_dict=col_dict, device=device)
        #train_loader = DataLoader(x_test_loader, shuffle=True, batch_size=4)
        #x_val_loader = IDSWDataSetLoader(x_val, y_val, col_dict=col_dict, device=device)
        #train_loader = DataLoader(x_val_loader, shuffle=True, batch_size=4)
        
        #model =smallnet3(in_chan=3, f_lin_lay=67968, l_lin_lay=11, ks=(3,5)).to(device) #10368
        vgg = nn.Sequential(
            #nn.Conv2d(1, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),#(0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            #PrintLayer(),
            model_vgg16,
            Squeeze(),
            nn.Linear(4096,11), # change to 10
            nn.Softmax(dim=0),
        )
    
        vgg.to(device)
        model=vgg
        
        if config.loss_fn == 'MSE':
            loss_fn = nn.MSELoss()
        elif config.loss_fn == 'CrossEntropy':
            loss_fn = nn.CrossEntropyLoss()

        e_count = 0
         # *

        optimizer = build_optimizer(model, config.optimizer, config.learning_rate, config.weight_decay)

        for epoch in tqdm(range(config.epochs)):
            # current_loss, predict_list, num_correct, label_list, model, optimizer
            #t_loss, t_predict_list_, t_num_correct, t_label_list_, model, optimizer = batch_loop(model, train_loader, epoch, loss_fn, device, col_dict, num_classes=11, optimizer=optimizer)
            t_loss, t_predict_list_, t_num_correct, t_label_list_, model, optimizer = res_loop(model, x_train, y_train, epoch, loss_fn, device, col_dict, num_classes=11, optimizer=optimizer)

            t_accuracy = (t_num_correct /len(x_train))*100
            t_loss_list.append(t_loss)
            t_predict_list.append(t_predict_list_)
            t_label_list.append(t_label_list_)
            t_accuracy_list.append(t_accuracy)

            #v_loss, v_predict_list_, v_num_correct, v_label_list_= batch_loop(model, x_val_loader, epoch, loss_fn, device,col_dict,num_classes=11, train=False)
            v_loss, v_predict_list_, v_num_correct, v_label_list_= res_loop(model, x_val, y_val, epoch, loss_fn, device,col_dict,num_classes=11, train=False)

            v_accuracy= (v_num_correct / len(x_val))*100
            v_loss_list.append(v_loss)
            v_predict_list.append(v_predict_list_)
            v_label_list.append(v_label_list_)
            v_accuracy_list.append(v_accuracy)

            t_avg_loss =t_loss/len(x_train)
            v_avg_loss = v_loss /len(x_val)

            e_count +=1
            # logging
            wandb.log({'avg_train_loss': t_avg_loss, 'epoch':epoch})
            wandb.log({'avg_val_loss': v_avg_loss, 'epoch':epoch})

            wandb.log({'train_loss': t_loss, 'epoch':epoch})
            wandb.log({'val_loss': v_loss, 'epoch':epoch})

            wandb.log({'train_correct': t_num_correct, 'epoch':epoch})
            wandb.log({'val_correct': v_num_correct, 'epoch':epoch})

            wandb.log({'train_accuracy_%': t_accuracy, 'epoch':epoch})
            wandb.log({'val_accuracy_%': v_accuracy, 'epoch':epoch})

            wandb.log({'t_labels': t_label_list, 'epoch':epoch})
            wandb.log({'v_labels': v_label_list, 'epoch':epoch})

            wandb.log({'t_predictions': t_predict_list, 'epoch':epoch})
            wandb.log({'v_predictions': v_predict_list, 'epoch':epoch})

            # add lists to save dict after all epochs run
    save_dict['Current_Epoch'] = config['epochs']
    save_dict['training_samples'] = len(x_train)# should this be the whole list for future graphs...?
    save_dict['validation_samples'] = len(x_val)
    save_dict['t_loss_list'] = t_loss_list #[c.to('cpu') for c in t_loss_list]
    save_dict['t_predict_list'] = [[c.to('cpu') for c in k]for k in t_predict_list] #[[c.to('cpu') for c in k]for k in t_predict_list]  # [c.to('cpu') for c in t_predict_list] 
    save_dict['t_accuracy_list'] = t_accuracy_list #
    save_dict['v_loss_list'] = v_loss_list #[c.to('cpu') for c in v_loss_list]
    save_dict['v_predict_list'] = [[c.to('cpu') for c in k]for k in v_predict_list]#[[c.to('cpu') for c in k]for k in v_predict_list] # [c.to('cpu') for c in v_predict_list]
    save_dict['v_accuracy_list'] = v_accuracy_list #
    save_dict['t_labels'] = t_label_list #[[c.to('cpu') for c in k]for k in t_label_list]
    save_dict['v_labels'] = v_label_list #[[c.to('cpu') for c in k] for k in v_label_list]
    
    title = save_dict['Run']
    #test_predictions, test_y, test_accuracy =           model, X, Y,         loss_fn, device, col_dict,title, num_classes
    test_predictions, test_y, test_accuracy = test_loop_res(model, x_test, y_test, loss_fn, device, col_dict, title,num_classes=11)

    save_dict['test_predictions']= [c.to('cpu') for c in test_predictions]
    save_dict['test_labels'] = test_y
    save_dict['test_acc'] = test_accuracy

    
    with open(f"/its/home/nn268/antvis/antvis/optics/pickles/{title}.pkl", 'wb+') as f:
        pickle.dump(save_dict, f)
        
    return model

In [17]:
# this run. works. crashed because of printing gradients.

In [18]:
#model,save_dict = pipeline(config) #7,168
model = train(device,col_dict, save_dict, config=config)
"""def tr(config=None):
    with wandb.init(config=config):
        config = wandb.config
        model, save_dict = pipeline(config)
        

wandb.agent(sweep_id, tr, count=2)"""

  0%| | 0/120 [00:00<?,
Traceback (most recent call last):
  File "/tmp/ipykernel_539171/3772374831.py", line 52, in train
    t_loss, t_predict_list_, t_num_correct, t_label_list_, model, optimizer = res_loop(model, x_train, y_train, epoch, loss_fn, device, col_dict, num_classes=11, optimizer=optimizer)
  File "/its/home/nn268/antvis/antvis/optics/loop_fns.py", line 115, in res_loop
    prediction = model.forward(tense) # tense
  File "/its/home/nn268/.local/lib/python3.10/site-packages/torch/nn/modules/container.py", line 215, in forward
    input = module(input)
  File "/its/home/nn268/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/its/home/nn268/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/its/home/nn268/.local/lib/python3.10/site-packages/torchvision/models/vgg.py", line 66, in forward

RuntimeError: Given input size: (512x1x8). Calculated output size: (512x0x4). Output size is too small

In [ ]:
!nvidia-smi

In [ ]:
# confusion matrix

import seaborn as sns

def plot_confusion(predictions:list, actual:list, title:str):
    predict_list = [int(t.argmax()) for t in predictions]
    actual = [int(l.argmax()) for l in actual]

    actual = np.array(actual)
    predict_list = np.array(predict_list)


    #FixedLocator locations (3), usually from a call to set_ticks, does not match the number of labels (11).
    print(f'\n     {title}')
    train_epoch_matrix = confusion_matrix(actual, predict_list, labels= [0,1,2,3,4,5,6,7,8,9,10])
    disp= ConfusionMatrixDisplay(train_epoch_matrix, display_labels=[0,1,2,3,4,5,6,7,8,9,10])
    disp.plot()
    plt.show()

def sns_heatmap(pred, actual, title): #Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead
    pred = [int(i.argmax().detach().numpy()) for i in pred]
    actual = [int(i.argmax().detach().numpy()) for i in actual]
    data = {'prediction':pred, 'actual':actual}
    data = pd.DataFrame(data)
    #data = (pred,actual)
    f, ax = plt.subplots(figsize=(9,6))
    sns.heatmap(data, cmap=sns.color_palette('pastel'), annot=True, cbar=True, ax=ax)
    

In [ ]:
save_dict.keys()
print(save_dict['Run'])

In [ ]:
print(type(t_predict))
#[print(type(i)) for i in t_predict]
import pandas as pd 
pred = [int(i.argmax().detach().numpy()) for i in t_predict]
actual = [int(i.argmax().detach().numpy() )for i in t_labels]
#print(len(pred))
#print(len(actual))

for i in pred:
    print(type(i))
    break
#df = {'prediction': pred, 'actual': actual}
#df = pd.DataFrame(df)
#print(df[:3])

In [ ]:
t_predict = save_dict['t_predict_list']
t_labels = save_dict['t_labels']

v_predict = save_dict['v_predict_list'] # WHY IS THERE NOTHING IN V OREDICT LIST!
v_labels = save_dict['v_labels']

#plot_confusion(t_predict, t_labels, 'Train Confusion Matrix', )
#plot_confusion(v_predict, v_labels, 'Validation Confusion Matrix')

In [ ]:
sns_heatmap(t_predict, t_labels, title='sns heat')

In [ ]:
"""def pipeline(config): 
    
    vgg = nn.Sequential(
            #nn.Conv2d(1, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),#(0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            #PrintLayer(),
            model_vgg16,
            Squeeze(),
            nn.Linear(4096,10), # change to 10
            nn.Softmax(dim=0),
        )
    title = f'IDSW_VGG16_fine_MNIST_test_122023'
    save_dict = {'Run' : title,
                 'Current_Epoch': 0,
save_location : r'pickles/'}
    
    vgg.to(device)
    
    #!nvidia-smi
    
    loss_list=[]
    #loss_fn = nn.CrossEntropyLoss()
    #loss_fn = nn.MSELoss()
    with wandb.init(project=title, config=config):
        config = wandb.config
        x_train,y_train, x_val, y_val,  x_test, y_test = preprocessMNIST()
        data= (x_train,y_train, x_val, y_val,  x_test, y_test)
        
    
        model = vgg
        optimizer = build_optimizer(model, config.optimizer, config.learning_rate, config.weight_decay)
                                    # 'seed', 'model', 'loss_fn', 'optimizer', and 'scheduler'
        model, save_dict = hp_sweep(config, col_dict,save_dict, device, best_acc=0, data=data, seed=5, model=model, loss_fn ='CrossEntropy', optimizer = optimizer, scheduler=0.2) #train_model(model,x_train, x_val, y_train, y_val, config) #train_dl, val_dl

    return model, save_dict"""

In [ ]:
"""model =nn.Sequential(
    PrintLayer(),
    model_vgg16,
    PrintLayer(),
    Squeeze(),
    PrintLayer(),
    nn.Linear(4096,11),
    PrintLayer(),
    nn.Softmax(dim=0),
    PrintLayer()

).to(device)"""

In [ ]:

"""
def train_model(model,  x_train, x_val, y_train, y_val, config, best_acc=0): #train_dl, val_dl, 
    #wandb.watch(model, log='all', log_freq=10)
    
    loss_fn = set_lossfn(config.loss_fn) # ****
    
    lr = config['learning_rate'] #1e-5 #config.learning_rate
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)#build_optimizer(model, optimizer=torch.optim.Adam(model.parameters(), lr=lr))#config.optimizer, config.learning_rate, config.weight_decay)
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config.scheduler, last_epoch=-1) #gamma=config.scheduler, last_epoch=-1)
                                                                    #scheduler'
    ####

    #model = model.to(device)
    model.train()
    
    
    #losses= []
    #predictions = []
    t_loss_list = []
    v_loss_list = []
    t_predict_list = []
    v_predict_list = []
    t_accuracy_list = []
    v_accuracy_list = []
    t_label_list = []
    v_label_list = []
    #labels = []
    
    total_epochs = 0
    for epoch in tqdm(range(config['epochs'])): #config.epochs)):
        print('E   ', epoch)
        t_correct = 0
        v_correct = 0
    
        if epoch == 0:
            #model = model.to('cpu')
            best_model = deepcopy(model)#.to(device)
            #model = model.to(device)
            
            
        #train_ids = random.shuffle(train_ids)
        #print(type(train_ids))
        print('training...')
        for idx, img in enumerate(x_train): 
            model.train()
            x = preprocess_im(img)
            #x = x.to(device)
            train_prediction = model.forward(x)
            
            y = label_oh_tf(y_train[idx], device=device, num_classes=10) # use same index val to index y (labels) and turn into onehot encoded label

            """
            #if idx % 1000 == 0:
            #print(idx, ' / ', len(x_train))
            #!nvidia-smi
            """
            #print(train_prediction, train_label)
        
        
            t_loss = loss_fn(train_prediction, y)

            if train_prediction.argmax() == y.argmax():
                t_correct+=1

            
            optimizer.zero_grad()
            t_loss.backward()
            
            if epoch == config['epochs']-1:
                plot_grad_flow(model)
            
            optimizer.step()
            scheduler.step()
            
            t_loss_list.append(t_loss.to('cpu'))
            t_predict_list.append(train_prediction.to('cpu'))
            t_label_list.append(y.to('cpu'))
            
        train_acc = (t_correct / len(x_train))
        t_accuracy_list.append(train_acc)
            
            
            
        print('validating...')
        
        for idx, img in enumerate(x_val):
            model.eval()
            x = preprocess_im(img)
            #x = x.to(device)
            val_prediction = model.forward(x)
            
            y = label_oh_tf(y_val[idx], device=device, num_classes=10)
            """
            #if idx % 100 == 0:
            #    print(idx, ' / ', len(x_val))
            #    !nvidia-smi
            """

            v_loss = loss_fn(val_prediction, y)
            
            if val_prediction.argmax() == y.argmax():
                v_correct +=1
                

            v_loss = v_loss.to('cpu')
            v_loss_list.append(v_loss.item())
            
            
            val_prediction = val_prediction.to('cpu')
            v_predict_list.append(val_prediction.detach().numpy())
            
            v_label_list.append(y.to('cpu').detach().numpy())
            
        val_acc = (v_correct / len(y_val))
        v_accuracy_list.append(val_acc)

        wandb.log({'train_loss': t_loss, 'epoch':epoch})
        wandb.log({'val_loss': v_loss, 'epoch':epoch})
        wandb.log({'train_accuracy_%': train_acc, 'epoch':epoch})
        wandb.log({'val_accuracy_%': val_acc, 'epoch':epoch})
            
        total_epochs += epoch

        if val_acc > best_acc:
    
            #print('Start Of SAve -----------------------------')
            #!nvidia-smi

            best_acc = val_acc
            
            #model = model.to('cpu')
            best_model = deepcopy(model)
            best_model = best_model.to('cpu')
            

            
            save_dict['Current_Epoch'] += config['epochs']
            save_dict['training_samples'] = len(x_train)
            save_dict['validation_samples'] = len(x_val)
            save_dict['t_loss_list'] = t_loss_list
            save_dict['t_predict_list'] = t_predict_list  
            save_dict['t_accuracy_list'] = t_accuracy_list  #
            save_dict['v_loss_list'] = v_loss_list
            save_dict['v_predict_list'] = v_predict_list  #
            save_dict['v_accuracy_list'] = v_accuracy_list  #
            save_dict['t_labels'] = t_label_list
            save_dict['v_labels'] = v_label_list
            
            model_architecture = [nn.Sequential(
                            model_vgg16,
                            Squeeze(),
                            nn.Linear(4096,10),
                            nn.Softmax(dim=0),
                        )]

            save_dict['model.state_dict'] = model.state_dict() #to('cpu').
            save_dict['model_architecture_untrained'] = model_architecture

            title = save_dict['Run']
            with open(f'{save_location}{title}.pkl', 'wb+') as f:
                pickle.dump(save_dict, f)
            
            print('improvment in metrics. model saved')

            #print('END Of SAve -----------------------------')
            #!nvidia-smi

        

        #if (epoch+1)%2==0:
            #train_log(t_loss, v_loss, epoch)
            #wandb.log({'train_accuracy_%': train_acc, 'epoch':epoch})
            #wandb.log({'val_accuracy_%': val_acc, 'epoch':epoch})
            
    model = best_model
    #labels = zip(t_label_list, v_label_list)
    #losses = zip(t_loss_list, v_loss_list)
    #predictions = zip(t_predict_list, v_predict_list)
    return model,save_dict



def pipeline(config): 
    
    vgg = nn.Sequential(
            #nn.Conv2d(1, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1)),#(0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            #PrintLayer(),
            model_vgg16,
            Squeeze(),
            nn.Linear(4096,10), # change to 10
            nn.Softmax(dim=0),
        )
    
    #model_vgg16.to('cpu')
    
    vgg.to(device)
    
    #!nvidia-smi
    
    loss_list=[]
    #loss_fn = nn.CrossEntropyLoss()
    #loss_fn = nn.MSELoss()
    with wandb.init(project=title, config=config):
        config = wandb.config
        x_train,y_train, x_val, y_val,  x_test, y_test = preprocessMNIST()
        data= (x_train,y_train, x_val, y_val,  x_test, y_test)
    
        model = vgg

        model, save_dict = hp_sweep(config, col_dict=None,save_dict, device, best_acc=0, data=data) #train_model(model,x_train, x_val, y_train, y_val, config) #train_dl, val_dl

return model, save_dict"""


In [ ]:

vgg = nn.Sequential(
        model_vgg16,
        Squeeze(),
        nn.Linear(4096,11),
        nn.Softmax(dim=0),
    )

#model_vgg16.to('cpu')

vgg.to(device)
next(vgg.parameters()).is_cuda

In [ ]:

x = preprocess_im(x_train[0])


train_prediction = model.forward(x)

In [ ]:
model.to('cpu').state_dict()